In [0]:
%sh apt-get update && apt-get install -y gdal-bin libeccodes-dev libnetcdf-dev libhdf5-dev

In [0]:

!pip install numpy==1.24.4 eccodes==1.7.1 xarray==2024.9.0 cf_xarray==0.10.0 cfgrib==0.9.14.1 netcdf4==1.7.2
!pip install --no-cache-dir --force-reinstall cfgrib==0.9.14.1 numpy==1.24.4

In [0]:
path_sandbox = "/Volumes/cdp_prd_sandbox_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/"
path_staging = "/Volumes/cdp_prd_staging_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/"
path_live = "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/"

In [0]:
import xarray as xr

import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def list_all_files(path):
    files = []
    for item in dbutils.fs.ls(path):
        if item.isDir():
            files.extend(list_all_files(item.path))
        else:
            # Skip .idx and .log files
            if not (item.path.endswith('.idx') or item.path.endswith('.log')):
                files.append(item.path)
    return files

def filter_files(file_list):
    filtered_file_list = []
    for f in file_list:
        try:
            if int(f.split('/')[-3]) > 15:
                filtered_file_list.append(f)
        except Exception as ex:
            if "repeated_APCP" in str(ex):
                pass
            else:
                raise Exception(ex)

    return filtered_file_list

def test_tcc_dataset(input_file_path: str) -> int:
    """Open a GRIB file using cfgrib and return an xarray Dataset"""

    try:
        dataset = xr.open_dataset(
            input_file_path,
            engine="cfgrib",
            backend_kwargs={
                "indexpath": "",
                "filter_by_keys": {"shortName": "tcc"},
            },
        )
        if not dataset:
            # Empty dataset
            return 3
        return 0
    except Exception as e:
        msg = str(e)
        if "multiple values for unique key" in msg and "filter_by_keys={'stepType': 'instant', 'shortName': 'tcc'}" in msg:
            return 1
        else:
            raise Exception(e)

def validate_files(file_list):
    n = len(file_list)
    data = list()
    columns = ['type', 'year', 'month', 'day', 'hour', 'filename', 'flag']  # adjust as needed

    for i, file in enumerate(file_list):
        file_name = file.replace("dbfs:", "")
        res = test_tcc_dataset(file_name)
        if res == 3:
            continue
        record = file_name.split("/")[-6:]
        record.append(res)
        data.append(record)
        if i % 10 == 0 and data:
            print((i+1)/n)
            # Append to Delta table
    #         df = spark.createDataFrame(data, columns)
    #         df.write.format("delta").mode("append").save(delta_table_path)
    #         data = []  # Clear the buffer

    # # Write any remaining records
    # if data:
    #     df = spark.createDataFrame(data, columns)
    #     df.write.format("delta").mode("append").save(delta_table_path)

    
    return data

def plot_validation_results(data, title='Flag Over Time'):
    df = pd.DataFrame(data, columns=['type', 'year', 'month', 'day', 'hour', 'filename', 'flag'])
    df['datetime'] = pd.to_datetime(df['year'] + df['month'] + df['day'] + df['hour'], format='%Y%m%d%H')

    plt.scatter(df['datetime'], df['flag'])
    plt.xlabel('Date')
    plt.ylabel('Flag')
    plt.title(title)
    plt.ylim(-0.1, 1.1)
    plt.yticks([0, 1])
    xmin = df['datetime'].min()
    xmax = df['datetime'].max()
    if xmin != xmax:
        plt.xlim(xmin, xmax)
    plt.show()

def find_switch_points(data):
    columns = ['type', 'year', 'month', 'day', 'hour', 'filename', 'flag']
    df = pd.DataFrame(data, columns=columns)
    # Create datetime column
    df['datetime'] = pd.to_datetime(df['year'] + df['month'] + df['day'] + df['hour'], format='%Y%m%d%H')
    df['flag_shift'] = df['flag'].shift(1)
    switch_points = df[df['flag'] != df['flag_shift']]
    return switch_points[['datetime', 'flag_shift', 'flag']]

In [0]:
#sandbox_files = filter_files(list_all_files(path_sandbox))
staging_files = filter_files(list_all_files(path_staging))
#live_files = filter_files(list_all_files(path_live))

#print(len(sandbox_files))
print(len(staging_files))
#print(len(live_files))

In [0]:
staging_files

In [0]:
#sandbox_data = validate_files(sandbox_files)
staging_data = validate_files(staging_files)
#live_data = validate_files(live_files)

In [0]:
find_switch_points(staging_data)

In [0]:
#plot_validation_results(sandbox_files, "Sandbox")
plot_validation_results(staging_data, "Staging")
#plot_validation_results(live_files, "Live")

In [0]:
columns = ['type', 'year', 'month', 'day', 'hour', 'filename', 'flag']
df = pd.DataFrame(staging_data, columns=columns)

# Group by year, month, day, hour and count 1s and 0s
result = df.groupby(['year', 'month', 'day', 'hour'])['flag'].value_counts().unstack(fill_value=0)

print(result)

In [0]:
dataset = xr.open_dataset(
    "/Volumes/cdp_prd_sandbox_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/19/18/gfs.t18z.pgrb2.0p25.f174",
    engine="cfgrib",
    backend_kwargs={
        "indexpath": "",
        "filter_by_keys": {"shortName": "tcc"},
    },
)

In [0]:
dataset = xr.open_dataset(
    "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/19/18/gfs.t18z.pgrb2.0p25.f174",
    engine="cfgrib",
    backend_kwargs={
        "indexpath": "",
        "filter_by_keys": {"shortName": "tcc"},
    },
)

In [0]:
dataset = xr.open_dataset(
    "/Volumes/cdp_prd_staging_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/25/06/gfs.t06z.pgrb2.0p25.f000",
    engine="cfgrib",
    backend_kwargs={
        "indexpath": "",
        "filter_by_keys": {"shortName": "tcc", 'stepType': 'instant'},
    },
)

In [0]:
dataset = xr.open_dataset(
    "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/25/12/gfs.t12z.pgrb2.0p25.f036",
    engine="cfgrib",
    backend_kwargs={
        "indexpath": "",
        "filter_by_keys": {"shortName": "tcc"},
    },
)

In [0]:
import os

# fails in staging:
file_size = os.path.getsize("/Volumes/cdp_prd_staging_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/25/06/gfs.t06z.pgrb2.0p25.f036")
print(file_size)

# succeeds in live
file_size = os.path.getsize("/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/25/06/gfs.t06z.pgrb2.0p25.f036")
print(file_size)

In [0]:
# succeeds in staging
file_size = os.path.getsize("/Volumes/cdp_prd_staging_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/20/12/gfs.t12z.pgrb2.0p25.f033")
print(file_size)

# succeeds in live
file_size = os.path.getsize("/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/20/12/gfs.t12z.pgrb2.0p25.f033")
print(file_size)

In [0]:
dataset = xr.open_dataset(
    "/Volumes/cdp_prd_staging_catalog_01/weather_bronze/bronze/noaa/GFS_pgrb2_0p25/2025/06/20/12/gfs.t12z.pgrb2.0p25.f033",
    engine="cfgrib",
    backend_kwargs={
        "indexpath": "",
        "filter_by_keys": {"shortName": "tcc"},
    },
)

In [0]:
print(dataset["tcc"])